In [1]:
!pip3 uninstall -y python-circos
%cd /Users/hideto/dropbox/HIDETO_MORI.LAB/Experiments/Project/pyCircos/pyCircos
!python3 setup.py install

Found existing installation: python-circos 0.2.0
Uninstalling python-circos-0.2.0:
  Successfully uninstalled python-circos-0.2.0
/Users/hideto/Dropbox (Yachie Lab)/HIDETO_MORI.LAB/Experiments/Project/pyCircos/pyCircos
running install
/usr/local/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
writing python_circos.egg-info/PKG-INFO
writing dependency_links to python_circos.egg-info/dependency_links.txt
writing requirements to python_circos.egg-info/requires.txt
writing top-level names to python_circos.egg-info/top_level.txt
reading manifest file 'python_circos.egg-info/SOURCES.txt'
adding license fil

In [2]:
import pycircos
import matplotlib.pyplot as plt
Tarc    = pycircos.Tarc
Tcircle = pycircos.Tcircle

In [3]:
import os
print(os.getcwd())
tarc    = Tarc(tree="./tutorial/sample_data/kegg/kegg.nwk", format="newick", interspace=1)
tcircle = Tcircle(figsize=(12,12)) 

/Users/hideto/Dropbox (Yachie Lab)/HIDETO_MORI.LAB/Experiments/Project/pyCircos/pyCircos


<Figure size 576x576 with 0 Axes>

In [4]:
#Set colors for terminal clades
phyla_names  = ["Actinobacteria","Aquificae","Bacteroidetes", "Chlamydiae","Chlorobi","Chloroflexi","Crenarchaeota", 
                "Cyanobacteria","Euryarchaeota","Firmicutes", "Spirochaetes", "Proteobacteria", "Tenericutes","Thermi","Thermotogae","Other"]
phyla_colors = ["#9ACD32", "#EE6A50", "#87CEFA", "#FFC125", "#D15FEE", "#8DEEEE", "#800000","#006400", "#800080", 
                "#808080", "#B0171F", "#B0171F", "#191970", "#7B68EE", "#00CD00", "#000000"]
pylum_color_dict = dict(zip(phyla_names, phyla_colors))

cladevisual_dict  = {}
phylum_clade_dict = {} 
with open("./tutorial/sample_data/kegg/tippoint_attr.csv") as f:
    f.readline() 
    for line in f:
        line = line.rstrip().split(",")
        if line[1] in pylum_color_dict:
            cladevisual_dict[line[0]] = {"color":pylum_color_dict[line[1]], "size":12, "linewidth":0.1, "edgecolor":"#303030"}
        else:
            cladevisual_dict[line[0]] = {"color":pylum_color_dict["Other"], "size":12, "linewidth":0.1, "edgecolor":"#303030"}

In [5]:
tcircle.add_tarc(tarc)
tcircle.set_tarcs()
tcircle.plot_tree(tarc.arc_id, rlim=(0,550), cladevisual_dict=cladevisual_dict, linewidth=0.4, linecolor="#606060")

In [6]:
tcircle.figure.savefig("tree-1.pdf")

In [7]:
os.getcwd()

'/Users/hideto/Dropbox (Yachie Lab)/HIDETO_MORI.LAB/Experiments/Project/pyCircos/pyCircos'

In [8]:
#Set highlights for specific clades
highlight_dict={"Staphylococcus" : {"color":"#808080", "alpha":0.3, "fontsize":5, "label":"Staphylococcus"}, 
                "Burkholderia"   : {"color":"#B0171F", "alpha":0.3, "fontsize":5, "label":"Burkholderia"},
                "Mycoplasma"     : {"color":"#191970", "alpha":0.3, "fontsize":5, "label":"Mycoplasma"}, 
                "Synechococcus"  : {"color":"#9ACD32", "alpha":0.3, "fontsize":5, "label":"Synechococcus"}}
tcircle.plot_highlight(tarc.arc_id, highlight_dict)

In [9]:
tcircle.figure.savefig("tree-2.pdf")

In [10]:
#Draw the first rings representing the genetic profiles of ATPases.
from matplotlib.colors import LinearSegmentedColormap
VA_ATP_rings = [{} for _ in range(8)]
F_ATP_rings  = [{} for _ in range(8)]
with open("./tutorial/sample_data/kegg/firstring_attr.csv") as f:
    f.readline() 
    for line in f:
        line = line.rstrip().split(",") 
        if int(line[1]) <= 8:
            VA_ATP_rings[int(line[1])-1][line[0]] = 1 
        else:
            F_ATP_rings[int(line[1])-9][line[0]]  = 1

In [11]:
cmap = LinearSegmentedColormap.from_list("tmp1", ["#FFFFFF","#dfac03"], N=2)
for i, VA_ATP_ring in enumerate(VA_ATP_rings):
    values = []
    for cladename in tarc.terminal_dict:
        VA_ATP_ring.setdefault(cladename, 0)
        values.append(VA_ATP_ring[cladename])
    tcircle.heatmap(tarc.arc_id, data=values, raxis_range=(550+5*i, 550+5*(i+1)), cmap=cmap)

In [12]:
cmap = LinearSegmentedColormap.from_list("tmp2", ["#FFFFFF","#339933"], N=2)
for i, F_ATP_ring in enumerate(F_ATP_rings):
    values = []
    for cladename in tarc.terminal_dict:
        F_ATP_ring.setdefault(cladename, 0)
        values.append(F_ATP_ring[cladename])
    tcircle.heatmap(tarc.arc_id, data=values, raxis_range=(590+5*i, 590+5*(i+1)), cmap=cmap)

In [13]:
tcircle.figure.savefig("tree-3.pdf")

In [14]:
#Draw the second rings representing the profiles of abundances of fatty acid metabolism.
secondring_dict = {"FA synth init":{}, "FA synth elong":{}, "acyl-CoA synth":{}, "beta-Oxidation":{}, "Ketone biosynth":{}}
with open("./tutorial/sample_data/kegg/secondring_attr.csv") as f:
    f.readline()
    for line in f:
        line = line.rstrip().split(",") 
        secondring_dict[line[2]][line[0]] = float(line[1])

In [15]:
cmap1 = LinearSegmentedColormap.from_list("atp1", ["#FFFFFF","#793a07"], N=500)
cmap2 = LinearSegmentedColormap.from_list("atp2", ["#FFFFFF","#9f1f9f"], N=500)
cmap3 = LinearSegmentedColormap.from_list("atp3", ["#FFFFFF","#0000be"], N=500)
cmap4 = LinearSegmentedColormap.from_list("atp4", ["#FFFFFF","#005500"], N=500)
cmap5 = LinearSegmentedColormap.from_list("atp5", ["#FFFFFF","#b22222"], N=500)
cmaps = [cmap1, cmap2, cmap3, cmap4, cmap5]
for i, key in enumerate(("Ketone biosynth", "beta-Oxidation", "acyl-CoA synth", "FA synth elong", "FA synth init")):
    values = []
    for cladename in tarc.terminal_dict:
        secondring_dict[key].setdefault(cladename, 0)
        values.append(secondring_dict[key][cladename])
    tcircle.heatmap(tarc.arc_id, data=values, raxis_range=(635+40*i, 635+40*(i+1)), cmap=cmaps[i])

In [16]:
tcircle.figure.savefig("tree-4.pdf")

In [17]:
#Draw the genome lengths
clade_genomelength_dict = {}
with open("./tutorial/sample_data/kegg/barplot_attr.csv") as f:
    f.readline() 
    for line in f:
        line = line.rstrip().split(",") 
        clade_genomelength_dict[line[0]] = float(line[1]) 

values     = [] 
facecolors = []
for cladename in tarc.terminal_dict:
    clade_genomelength_dict.setdefault(cladename, 0) 
    values.append(clade_genomelength_dict[cladename]) 
    facecolors.append(cladevisual_dict[cladename]["color"])
tcircle.barplot(tarc.arc_id, data=values, raxis_range=(840,990), facecolor=facecolors)

In [18]:
tcircle.figure.savefig("tree-5.pdf")

In [19]:
tcircle.figure.savefig("tree-5.png", dpi=300)